In [1]:
# Main libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from configs import *
from fetch_data import *
from data_shuffling_split import *

In [2]:
dialect_dataset = read_csv("dialect_dataset_preprocessed_without_Lemmatization.csv")

# Display first 5 rows from the file
dialect_dataset.head()

Number of instances in the file are:  458197


,id,dialect,text
0,1175358310087892992,IQ,حسابشخصي لكن بالنهايه . . ينتفض . . يغير .
1,1175416117793349632,IQ,حسابشخصي يعني هذا محسوب علي البشر . . حيونه وح...
2,1175450108898565888,IQ,حسابشخصي مبين من كلامه خليجي
3,1175471073770573824,IQ,حسابشخصي يسلملي مرورك وروحك الحلوه 💐
4,1175496913145217024,IQ,حسابشخصي وين هل الغيبه اخ محمد 🌺


In [3]:
train_set, test_set             = general_split_and_shuffle(dialect_dataset, "dialect", .02)
strat_train_set, strat_test_set = Stratified_split_and_shuffle(dialect_dataset, "dialect", .02)

The number of instances in the training data are:  449033
The number of instances in the testing data are:   9164
The number of instances in the training data are:  449033
The number of instances in the testing data are:   9164


In [4]:
compare_df = compare_random_and_stratified_split(dialect_dataset, train_set, test_set, strat_train_set, strat_test_set)

In [5]:
compare_df

,overall,stratified_test,random_test,stratified_train,random_train,stratified_test. %error,random_test. %error,stratified_train. %error,random_train. %error
0,0.020179,0.020188,0.019424,0.020179,0.020195,-0.000860,0.075526,0.000018,0.001541
1,0.021665,0.021606,0.022370,0.021667,0.021651,0.005907,0.070479,0.000121,0.001438
2,0.025183,0.025207,0.027717,0.025183,0.025132,-0.002384,0.253366,0.000049,0.005171
3,0.031502,0.031536,0.031646,0.031501,0.031499,-0.003471,0.014384,0.000071,0.000294
4,0.033822,0.033828,0.032628,0.033822,0.033797,-0.000633,0.119402,0.000013,0.002463
5,0.035319,0.035356,0.034701,0.035318,0.035331,-0.003687,0.061787,0.000075,0.001261
6,0.035448,0.035465,0.035028,0.035447,0.035505,-0.001723,0.041926,0.000035,0.005755
7,0.041720,0.041685,0.043103,0.041721,0.041692,0.003519,0.138340,0.000072,0.002823
8,0.057381,0.057399,0.056962,0.057381,0.057317,-0.001708,0.041941,0.000035,0.006493
9,0.057390,0.057399,0.058817,0.057390,0.057352,-0.000835,0.142695,0.000017,0.003803


In [ ]:
ls = [0.02017909, 0.02166535, 0.02518349, 0.03150173, 0.0338217,  0.03531887,
 0.03544763 ,0.04172005, 0.05738143, 0.05739016, 0.05855996, 0.0602732,
 0.06093667, 0.06780708 ,0.07965788 ,0.09190152 ,0.09546549, 0.12578869]
sum(ls)

In [ ]:

stratified_test.sort()
stratified_train.sort()
overall.sort()

labels = dialect_dataset['dialect'].value_counts(ascending=True)

keys = list(dicts.keys())

compare_dict = {'classes': keys, 'overall': overall, 'stratified_test': stratified_test, 'random_test':random_test , 
                'stratified_train': stratified_train, 'random_train': random_train}

compare_df = pd.DataFrame(compare_dict)


In [ ]:
keys

In [ ]:
len(stratified_train)

In [ ]:
# In table below we can see how stratified static method most similar to overall coverage of our dataset
# look also at the simple random Vs overall
compare_df['stratified_test. %error'] = np.abs((compare_df['stratified_test'] - compare_df['overall']) * 100)
compare_df['random_test. %error'] = np.abs((compare_df['random_test'] - compare_df['overall']) * 100)

compare_df['stratified_train. %error'] = np.abs((compare_df['stratified_train'] - compare_df['overall']) * 100)

compare_df['random_train. %error'] = np.abs((compare_df['random_train'] - compare_df['overall']) * 100)

compare_df

In [ ]:
dialect_dataset['dialect'].value_counts(ascending=True)

In [ ]:
dicts = dict(dialect_dataset['dialect'].value_counts())

In [ ]:
keys = dicts.keys()

In [ ]:
dicts = dict(sorted(dicts.items(), key=lambda item: item[1]))

In [ ]:
keys